# Local alignment with the Smith-Waterman algorithm

This notebook uses a simple implementation of local alignment methods to align the two short DNA sequences from the lecture discussion. 

For simplicity a very simple scoring scheme is defined instead of a complete scoring matrix or substitution table. 

Recall that unlike edit distance based methods the aim will be to *maximize* the score from this matrix applied to the sequence position comparisons. The scoring scheme includes a positive value for matching - distinct from edit-distance calculations where matching is the default and gains nothing. 

Non-matching options include the insertion of a gap into either *sequ* or *seqw* sequence. Gaps have negative scores and are equivalent to Deletions or Insertions in terms of editting. 

The first step is to populate the V matrix based on the scoring scheme and then the second step is the traceback. Traceback follows the maximum scoring positions in the matrix until it is terminated by a zero value.

The Python library NumPy is used for the matrix operations.

In [ ]:
# run this cell to import Python numpy library
import numpy

The following scheme defines a simple example scoring matrix: 1 for matched nucleotides, -1 for mismatched nucleotides, -1 for a gap inserted in either.

In [ ]:
def scoringScheme(un, wn):
    '''Scoring scheme: 1 for match, -1 for gap, else -1 (for mismatch) '''
    if un == wn: return 1 # matching nucleotides
    if un == '-' or wn == '-': return -1 # gap
    return -1

Here are the two DNA sequences from the lecture example: 'GGTATGCTGGCGC' and 'TATATGCGGCGTTT'. As the first of these is shorter we add a gap to make them equal lengths. 

In [ ]:
seqa, seqb = '-GGTATGCTGGCGC', 'TATATGCGGCGTTT'

To see how similar the complete sequences are write a cell to apply the scoringScheme function to the sequences one nucleotide at a time. 

In [ ]:
# write your Python code to compare the sequences seqa and seqb

How well did this score? 
Can you improve on the score by changing the position of the gap - to the beginning of *seqa* or somewhere in the middle? 

The problem here is that the sequences do not globally align very well at all. However there is a shorter sequence motif that they have in common. A local alignment will help locate that.

# Smith-Waterman algorithm for optimal local alignment

The smithWaterman function below populates the V[*i,j*] matrix elements using the method described in the lecture. Notice that the indices *i*-1,*j*-1 refer to the current positions in the sequences as each has an appended initial empty string. The scoring used will be the scoring scheme function defined abover rather than a *similarity matrix*.  

The local alignment V[*i,j*] expression is the *maximum* form with the added '0' value (for an empty suffix) as explained in the lecture.

The function uses the numpy function *where* to return the *i* and *j* indices of the the best similarity score which is reported.

In [ ]:
def smithWaterman(u, w, scoring):
    ''' Fills V matrix values for sequences u and w using
        dynamic programming. Returns matrix and maximum alignment score.'''
    V = numpy.zeros((len(u)+1, len(w)+1), dtype=int) 
    for i in range(1, len(u)+1):
        for j in range(1, len(w)+1):
            V[i, j] = max(V[i-1, j-1] + scoring(u[i-1], w[j-1]), # max used 
                          V[i-1, j  ] + scoring(u[i-1], '-'),    
                          V[i  , j-1] + scoring('-',    w[j-1]), 
                          0)                               
    scoremax = numpy.where(V == V.max()) #finds optimal alignment score
    return V, int(V[scoremax])

Here is the example alignment problem from the lecture for sequences seqa and seqb. The full matrix is printed out including the initial amended empty row and column.

In [ ]:
seqa, seqb = 'GGTATGCTGGCGC', 'TATATGCGGCGTTT'
V, opt = smithWaterman(seqa, seqb, scoringScheme)
print(V)
print("Optimum=%d, ends at %s" % (opt, numpy.unravel_index(numpy.argmax(V), V.shape)))

To retrieve the actual alignment, a traceback following the maximum V[*i,j*] values is required. The following function calls the scoringScheme again starting from the optimum. The edits transcript is constructed together with the alignment display. 

Notice that if the full matrix is retained in memory then the scoring function is not required. It is used here for generality as for searches of large numbers of sequences often only the V max score and its position are kept.  

In [ ]:
def traceback(V, u, w, scoring):
    ''' Traceback from optimum in the V matrix  '''
    # get i, j for optimum
    i, j = numpy.unravel_index(numpy.argmax(V), V.shape)
    edits, alignNu, alignNw, alignMark = [], [], [], []
    while (i > 0 or j > 0) and V[i, j] != 0:
        diagl, vertl, horizl = 0, 0, 0
        if i > 0 and j > 0:
            diagl = V[i-1, j-1] + scoring(u[i-1], w[j-1])
        if i > 0:
            vertl = V[i-1, j] + scoring(u[i-1], '-')
        if j > 0:
            horizl = V[i, j-1] + scoring('-', w[j-1])
        if diagl >= vertl and diagl >= horizl:
            match = u[i-1] == w[j-1]
            edits.append('M' if match else 'R')
            alignMark.append('|' if match else ' ')
            alignNu.append(u[i-1]); alignNw.append(w[j-1])
            i -= 1; j -= 1
        elif vertl >= horizl:
            edits.append('D')
            alignNu.append(u[i-1]); alignNw.append('-'); alignMark.append(' ')
            i -= 1
        else:
            edits.append('I')
            alignNw.append(w[j-1]); alignNu.append('-'); alignMark.append(' ')
            j -= 1
    # the edits transcript is built up in reverse
    edits = (''.join(edits))[::-1]
    # final alignment for display
    alignment = '\n'.join(map(lambda u: ''.join(u), [alignNu[::-1], alignMark[::-1], alignNw[::-1]]))
    return edits, alignment

Here is the editing transcript: M is a match and D is a deletion - equivalent to a gap insertion in the *other* sequence. We anticipate at least one deletion/gap or insertion/gap as we know the sequences differs in length by one nucleotide character.

In [ ]:
print(traceback(V, seqa, seqb, scoringScheme)[0])

And this is a representation of the local alignment (as in the lecture). You can see that the deletion edit is actually a gap insertion in the second sequence.

In [ ]:
print(traceback(V, u, w, scoringScheme)[1])

Here is an alignment with a second sequence that has a further mutation. 

In [ ]:
seqx = 'TATATGCCGCGTTT'
V, opt = smithWaterman(seqa, seqx, scoringScheme)
print(V)
print("Optimum=%d, ends at %s" % (opt, numpy.unravel_index(numpy.argmax(V), V.shape)))

The optimum score is decreased and the edits transcript now includes R for 'replacement' since there is a mismatch.

In [ ]:
print(traceback(V, seqa, seqx, scoringScheme)[0])

In [ ]:
print(traceback(V, seqa, seqx, scoringScheme)[1])

Optional question: Can you use your earlier code for a similrity calculation to confirm the value for this optimum local alignment? 